### from : 04.hyde_experiment.md

In [1]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_REGION = os.getenv("PINECONE_INDEX_REGION")
PINECONE_INDEX_CLOUD = os.getenv("PINECONE_INDEX_CLOUD")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_INDEX_METRIC = os.getenv("PINECONE_INDEX_METRIC")
PINECONE_INDEX_DIMENSION = int(os.getenv("PINECONE_INDEX_DIMENSION"))

print("환경 변수 로딩 완료")

환경 변수 로딩 완료


In [5]:
import pandas as pd

# 문서 및 질의 데이터 로드
documents_df = pd.read_csv("../../datas/documents.csv")
queries_df = pd.read_csv("../../datas/queries.csv")

print(f"문서 수: {len(documents_df)}")
print(f"질의 수: {len(queries_df)}")

문서 수: 30
질의 수: 30


In [6]:
from konlpy.tag import Okt
from rank_bm25 import BM25Okapi

# Mecab 형태소 분석기 초기화
mecab = Okt()
# 문서 토큰화
tokenized_docs = [mecab.morphs(content) for content in documents_df['content']]
bm25 = BM25Okapi(tokenized_docs)

def bm25_search(query, top_k=5):
    tokens = mecab.morphs(query)
    scores = bm25.get_scores(tokens)
    ranked_idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    return [documents_df['doc_id'].iloc[i] for i in ranked_idx[:top_k]]

In [7]:
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# Pinecone 연결 (기존 인덱스 사용)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# 임베딩 모델 준비 (OpenAI)
embedding_model = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY)
# Dense 벡터 스토어 초기화
vector_store = PineconeVectorStore(index_name=PINECONE_INDEX_NAME, embedding=embedding_model)

print("Dense Retrieval 설정 완료")

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dense Retrieval 설정 완료


In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 인스턴스 생성 (OpenAI LLM 모델 사용)
chat_model = ChatOpenAI(
    model_name=OPENAI_LLM_MODEL,
    openai_api_key=OPENAI_API_KEY,
    temperature=0.3
)

# HyDE용 프롬프트 템플릿 설정
hyde_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
질문: {query}
아래 질문에 대해 실제 문서가 아니어도 좋으니 짧게 가상의 답변을 생성하세요:
"""
)

# 출력 파서 설정
output_parser = StrOutputParser()

# 프롬프트 -> LLM -> 출력파서를 연결하여 체인 생성
hyde_chain = hyde_prompt | chat_model | output_parser

print("HyDE LCEL 파이프라인 구성 완료")

HyDE LCEL 파이프라인 구성 완료


In [9]:
import numpy as np

def parse_relevant(relevant_str):
    pairs = relevant_str.split(';')
    rel_dict = { }
    for pair in pairs:
        doc_id, grade = pair.split('=')
        rel_dict[doc_id] = int(grade)
    return rel_dict

def compute_metrics(predicted, relevant_dict, k=5):
    hits = sum(1 for doc in predicted[:k] if doc in relevant_dict)
    precision = hits / k
    total_relevant = len(relevant_dict)
    recall = hits / total_relevant if total_relevant > 0 else 0
    rr = 0
    for idx, doc in enumerate(predicted):
        if doc in relevant_dict:
            rr = 1 / (idx + 1)
            break
    num_correct = 0
    precisions = []
    for i, doc in enumerate(predicted[:k]):
        if doc in relevant_dict:
            num_correct += 1
            precisions.append(num_correct / (i + 1))
    ap = np.mean(precisions) if precisions else 0
    return precision, recall, rr, ap

def evaluate_all(results_dict, queries_df, k=5):
    prec_list, rec_list, rr_list, ap_list = [], [], [], []
    for idx, row in queries_df.iterrows():
        qid = row['query_id']
        relevant = parse_relevant(row['relevant_doc_ids'])
        predicted = results_dict[qid]
        p, r, rr, ap = compute_metrics(predicted, relevant, k)
        prec_list.append(p)
        rec_list.append(r)
        rr_list.append(rr)
        ap_list.append(ap)
    return {
        'P@5': np.mean(prec_list),
        'R@5': np.mean(rec_list),
        'MRR': np.mean(rr_list),
        'MAP': np.mean(ap_list)
    }

In [10]:
import time

# 질의마다 가상 답변 생성 (HyDE 체인 사용)
hyde_pseudo = {}
for idx, row in queries_df.iterrows():
    qid = row['query_id']
    query_text = row['query_text']
    # HyDE 체인으로 가상 답변 생성
    pseudo_answer = hyde_chain.invoke({"query": query_text})
    hyde_pseudo[qid] = pseudo_answer
    time.sleep(0.5)  # API rate limit 대비 약간의 지연

print("HyDE 가상 답변 생성 완료")

HyDE 가상 답변 생성 완료


In [11]:
# Dense Retrieval (원본 질의 사용)
dense_results = {}
for idx, row in queries_df.iterrows():
    qid = row['query_id']
    query_text = row['query_text']
    docs = vector_store.similarity_search(query_text, k=5)
    dense_results[qid] = [doc.metadata['doc_id'] for doc in docs]

# HyDE Retrieval (가상 답변 사용)
hyde_results = {}
for qid, pseudo in hyde_pseudo.items():
    docs = vector_store.similarity_search(pseudo, k=5)
    hyde_results[qid] = [doc.metadata['doc_id'] for doc in docs]

print("Dense 및 HyDE 검색 결과 저장 완료")

Dense 및 HyDE 검색 결과 저장 완료


In [12]:
import pandas as pd

# 평가 수행 (각각 P@5, R@5, MRR, MAP 계산)
dense_metrics = evaluate_all(dense_results, queries_df, k=5)
hyde_metrics = evaluate_all(hyde_results, queries_df, k=5)

# 데이터프레임으로 결과 정리
df = pd.DataFrame({
    'Metric': dense_metrics.keys(),
    'Dense': dense_metrics.values(),
    'HyDE': hyde_metrics.values()
})
df

,Metric,Dense,HyDE
0,P@5,0.466667,0.480000
1,R@5,1.716667,1.711111
2,MRR,0.977778,0.966667
3,MAP,0.978148,0.957778
